In [58]:
import pandas
import matplotlib.pyplot as plt
import statsmodels
import tsfresh as ts
import featuretools as ft
import woodwork.logical_types as logical_types
from sklearn.preprocessing import LabelEncoder

data = pandas.read_csv("train.csv")
data_features = data.drop(columns="num_sold")
data_targets = data.num_sold

entity_set = ft.EntitySet()
entity_set.add_dataframe(
    dataframe_name="Book Sales",
    dataframe=data_features,
    index="row_id",
    time_index="date",
    logical_types={
        "country": logical_types.Categorical,
        "store": logical_types.Categorical,
        "product": logical_types.Categorical
    }
)
feature_matrix, features = ft.dfs(entityset=entity_set, target_dataframe_name="Book Sales", max_depth=1)


/opt/anaconda3/lib/python3.8/site-packages/featuretools/synthesis/deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(


In [59]:
label_encoder = LabelEncoder()
feature_matrix.country = label_encoder.fit_transform(feature_matrix.country) + 1
feature_matrix.store = label_encoder.fit_transform(feature_matrix.store) + 1
feature_matrix["product"] = label_encoder.fit_transform(feature_matrix["product"]) + 1

In [60]:
training_data_features = feature_matrix[feature_matrix["YEAR(date)"] < 2020]
training_data_targets = data_targets[range(len(training_data_features))]

test_data_features = feature_matrix[feature_matrix["YEAR(date)"] >= 2020]
start_index = test_data_features.index[0]
end_index = test_data_features.index[-1] + 1
test_data_targets = data_targets[range(start_index, end_index)]